# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that we've discussed some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modelling pipeline.  

With this pipeline, yoiu will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modelling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. We've discussed the bias variance tradeoff, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modelling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

## Loading the Data

In [94]:
#Your code here; load and preview the dataset
import pandas as pd
import numpy as np


df = pd.read_csv('loan_final.csv')


df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

## Defining the Problem

Set up the problem by defining X and Y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Be sure to use dummy variables for categorical variables and to normalize numerical quanitities. Be sure to also remove any rows with null data.  

For Y, we are looking to build a model to predict the total payment received for a loan.

In [95]:
df.columns

Index(['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'addr_state',
       'total_acc', 'total_pymnt', 'application_type'],
      dtype='object')

In [96]:
# Your code here; appropriately define X and Y using dummy variables and normalization for preprocessing.


X = df.drop(['total_pymnt'], axis =1)
X = X.drop(['term', 'int_rate', 'grade', 'loan_status', 'purpose', 'addr_state', 'total_acc', 'application_type'], axis =1)

In [97]:
Y = df['total_pymnt']

In [98]:
Y.head()

0     5863.155187
1     1014.530000
2     3005.666844
3    12231.890000
4     4066.908161
Name: total_pymnt, dtype: float64

In [99]:
Y

0         5863.155187
1         1014.530000
2         3005.666844
3        12231.890000
4         4066.908161
5         5632.210000
6        10137.840010
7         3939.135294
8          647.500000
9         1484.590000
10        7678.017673
11       13947.989160
12        2277.320000
13        3480.269999
14        7471.990000
15        1270.716942
16       12527.150000
17        3785.271965
18        7167.072893
19        9460.000848
20       27679.050000
21       14034.600000
22       11907.350000
23       11540.891980
24        2055.850000
25       15823.999050
26           0.000000
27        1611.160000
28        4486.293519
29       10126.088440
             ...     
41364     1248.176248
41365     4716.301934
41366     1551.808247
41367     3976.397706
41368     2496.458018
41369    10075.193810
41370     1516.971229
41371      991.170000
41372     5015.060693
41373     1038.800000
41374     5418.479706
41375     1501.250000
41376     1204.076744
41377     1399.520086
41378    1

In [100]:
Y_df = pd.DataFrame(Y)

In [101]:
X.head()

,loan_amnt,funded_amnt_inv,installment,emp_length,home_ownership,annual_inc,verification_status
0,5000.0,4975.0,162.87,10+ years,RENT,24000.0,Verified
1,2500.0,2500.0,59.83,< 1 year,RENT,30000.0,Source Verified
2,2400.0,2400.0,84.33,10+ years,RENT,12252.0,Not Verified
3,10000.0,10000.0,339.31,10+ years,RENT,49200.0,Source Verified
4,3000.0,3000.0,67.79,1 year,RENT,80000.0,Source Verified


## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modelling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [103]:
from sklearn.preprocessing import StandardScaler

# X = X.dropna()

sc = StandardScaler()

X_numeraical = X[['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']]


X_scaled = sc.fit_transform(X_numeraical)


X_scaled = pd.DataFrame(X_scaled, columns = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc'])


In [104]:
dummy_data = pd.get_dummies(df[['emp_length', 'home_ownership', 'verification_status']])


In [105]:
data = pd.concat([X_scaled, dummy_data], axis=1)

In [106]:
data.describe()

,loan_amnt,funded_amnt_inv,installment,annual_inc,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,...,emp_length_9 years,emp_length_< 1 year,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,4.139400e+04,4.139400e+04,4.139400e+04,4.139400e+04,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,...,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000,41394.000000
mean,-8.582678e-17,-4.394331e-17,-2.317323e-16,-9.269292e-17,0.086776,0.226265,0.114558,0.105378,0.088153,0.083539,...,0.032372,0.121853,0.446611,0.000097,0.003286,0.073175,0.476832,0.441707,0.244142,0.314152
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,0.281510,0.418417,0.318491,0.307043,0.283521,0.276698,...,0.176988,0.327120,0.497147,0.009830,0.057226,0.260426,0.499469,0.496596,0.429582,0.464183
min,-1.437865e+00,-1.427717e+00,-1.478230e+00,-1.051289e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.771264e-01,-7.311963e-01,-7.533680e-01,-4.455002e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.568407e-01,-2.226430e-01,-2.145854e-01,-1.511573e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.173829e-01,5.498138e-01,5.092610e-01,2.089944e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,3.214277e+00,3.472431e+00,4.686278e+00,9.186971e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [107]:
data.isna().sum()

loan_amnt                              0
funded_amnt_inv                        0
installment                            0
annual_inc                             0
emp_length_1 year                      0
emp_length_10+ years                   0
emp_length_2 years                     0
emp_length_3 years                     0
emp_length_4 years                     0
emp_length_5 years                     0
emp_length_6 years                     0
emp_length_7 years                     0
emp_length_8 years                     0
emp_length_9 years                     0
emp_length_< 1 year                    0
home_ownership_MORTGAGE                0
home_ownership_NONE                    0
home_ownership_OTHER                   0
home_ownership_OWN                     0
home_ownership_RENT                    0
verification_status_Not Verified       0
verification_status_Source Verified    0
verification_status_Verified           0
dtype: int64

In [108]:
Y_scaled = sc.fit_transform(Y_df)

In [109]:
Y = pd.DataFrame(Y_scaled, columns = ['labels'])
Y.head()

,labels
0,-0.685762
1,-1.218038
2,-0.999454
3,0.013389
4,-0.882952


## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. Be sure your function returns performance metrics regarding the training and validation sets.

In [110]:
#Your code here; define a function to evaluate a model object using K folds cross validation.

# def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.1, random_state=123)

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model.   

In general, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. We set up a k-folds cross-validation for this task since the dataset is not overly large.  

Build an initial sequential model with 2 hidden relu layers. The first should have 7 hidden units, and the second 10 hidden units. Finally, add a third layer with a linear activation function to output our predictions for the total loan payment. 

In [111]:
#Your code here; define and compile an initial model as described
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    colors = sns.color_palette("Set2")

    validation_scores = [] 
    
    kf = KFold(n_splits=10, shuffle=True)
    
    fig, axes = plt.subplots(2, 5, figsize=(12,8))
    for i, (train_index, test_index) in enumerate(kf.split(features_train)):
        "Currently graph imaging assumes 10 folds and is hardcoded to 5x2 layout."
        row = i//5
        col = i%5
        X_train, X_val = features_train.iloc[train_index], features_train.iloc[test_index]
        y_train, y_val = labels_train.iloc[train_index], labels_train.iloc[test_index]
        
        model = model_obj
        hist = model.fit(X_train, y_train, batch_size=32,
                         epochs=n_epochs, verbose=0, validation_data = (X_val, y_val))
        #Note: verboxe=0 turns off printouts regarding training for each epoch.
        #Potential simpler methodology
        validation_score = model.evaluate(X_val, y_val)
        validation_scores.append(validation_score)
        ax = axes[row, col]
        k = 'val_loss'
        d = hist.history[k]
        ax.plot(d, label=k, color=colors[0])

        k = 'loss'
        d = hist.history[k]
        ax.plot(d, label=k, color=colors[1])
        ax.set_title('Fold {} Validation'.format(i+1))
    #Final Graph Formatting
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
    plt.legend(bbox_to_anchor=(1,1))
    
    #General Overview
    validation_score = np.average(validation_scores)
    print('Mean Validation Score:', validation_score)
    print('Standard Deviation of Validation Scores:', np.std(validation_scores))
    return validation_score

## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [ ]:
#Your code here; use your k-folds function to evaluate the baseline model.
from keras.models import Sequential
from keras.layers import Dense


np.random.seed(123)


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.  

For example, here are some timing notes on potential experiments run on a Macbook Pro 3.1 GHz Intel Core i5 with 16gb of RAM:

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 100 epochs took approximately 15 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 25 units per hidden layer and 100 epochs took approximately 25 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 250 epochs took approximately 45 minutes to train and validate


In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your expirements above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modelling process)!

In [ ]:
#Your code here; final model training on entire training set followed by evaluation on hold-out data

## Additional Resources

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

## Summary

In this lab, we investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. We began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, we implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, we'll begin to investigate other neural network architectures such as CNNs.